In [1]:
pip install flask psycopg2-binary flask-cors


In [3]:
!mkdir products_api
%cd products_api

/content/products_api


In [ ]:
from flask import Flask, jsonify, request
import psycopg2
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

# Database connection config
DB_CONFIG = {
    "dbname": "store_db",
    "user": "postgres",        # 🔹 change this
    "password": "SQL123", # 🔹 change this
    "host": "localhost",
    "port": "5432"
}

# Function to connect to DB
def get_connection():
    try:
        return psycopg2.connect(**DB_CONFIG)
    except Exception as e:
        print("DB Connection Error:", e)
        return None

# 1️⃣ Home route
@app.route('/')
def home():
    return jsonify({"message": "Welcome to the Products API"}), 200

# 2️⃣ Get all products with pagination
@app.route('/products', methods=['GET'])
def get_products():
    try:
        page = int(request.args.get('page', 1))
        limit = int(request.args.get('limit', 20))
        offset = (page - 1) * limit

        conn = get_connection()
        if not conn:
            return jsonify({"error": "Database connection failed"}), 500

        cur = conn.cursor(cursor_factory=RealDictCursor)
        cur.execute("SELECT id, name, category, brand, retail_price FROM products ORDER BY id LIMIT %s OFFSET %s;",
                    (limit, offset))
        products = cur.fetchall()
        cur.close()
        conn.close()

        if not products:
            return jsonify({"message": "No products found"}), 404

        return jsonify({"page": page, "limit": limit, "products": products}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# 3️⃣ Get single product by ID
@app.route('/products/<int:product_id>', methods=['GET'])
def get_product(product_id):
    try:
        conn = get_connection()
        if not conn:
            return jsonify({"error": "Database connection failed"}), 500

        cur = conn.cursor(cursor_factory=RealDictCursor)
        cur.execute("SELECT id, name, category, brand, retail_price FROM products WHERE id = %s;", (product_id,))
        product = cur.fetchone()
        cur.close()
        conn.close()

        if not product:
            return jsonify({"error": "Product not found"}), 404

        return jsonify(product), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# 4️⃣ Error handler for invalid routes
@app.errorhandler(404)
def not_found(error):
    return jsonify({"error": "Endpoint not found"}), 404

# 5️⃣ Error handler for server errors
@app.errorhandler(500)
def server_error(error):
    return jsonify({"error": "Internal server error"}), 500

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [4]:
%%writefile app.py
from flask import Flask, request, jsonify
from flask_cors import CORS
import psycopg2
import os

app = Flask(__name__)
CORS(app)

@app.route('/')
def home():
    return "Welcome to the Products API!"

if __name__ == '__main__':
    app.run(debug=True)

Writing app.py


In [ ]:
!python app.py

In [ ]:
{"message": "Welcome to the Products API"}
